In [2]:
import os
import sys

import branca.colormap as cm
import numpy as np
import pandas as pd
from shapely.geometry import LineString, Point, Polygon

sys.path.insert(0, "../src")
import json
import time as ti
import math

import geopandas as gpd
import seaborn as sns
from sklearn import preprocessing
from tqdm import tqdm
import seaborn as sns 
from IPython.display import display, Math, Latex
import requests

from BrusselsMap import BrusselsMap
from GeoJsonHandler import GeoJsonHandler
import urllib.request

pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)

ImportError: dlopen(/opt/homebrew/Caskroom/miniconda/base/envs/optim_env/lib/python3.10/site-packages/scipy/sparse/linalg/_isolve/_iterative.cpython-310-darwin.so, 0x0002): Library not loaded: @rpath/liblapack.3.dylib
  Referenced from: <78036CAA-82A0-3FC2-9DF1-A2F5970A489B> /opt/homebrew/Caskroom/miniconda/base/envs/optim_env/lib/python3.10/site-packages/scipy/sparse/linalg/_isolve/_iterative.cpython-310-darwin.so
  Reason: tried: '/opt/homebrew/Caskroom/miniconda/base/envs/optim_env/lib/python3.10/site-packages/scipy/sparse/linalg/_isolve/liblapack.3.dylib' (no such file), '/opt/homebrew/Caskroom/miniconda/base/envs/optim_env/lib/python3.10/site-packages/scipy/sparse/linalg/_isolve/../../../../../../liblapack.3.dylib' (no such file), '/opt/homebrew/Caskroom/miniconda/base/envs/optim_env/lib/python3.10/site-packages/scipy/sparse/linalg/_isolve/liblapack.3.dylib' (no such file), '/opt/homebrew/Caskroom/miniconda/base/envs/optim_env/lib/python3.10/site-packages/scipy/sparse/linalg/_isol

# Now let's try to compute the distance OD Matrix between these points

First, let's define the function need to perform the HTTP GET request. All the assumptions regarding this matrix are under this first link below.

- Resources: https://docs.microsoft.com/en-us/bingmaps/rest-services/examples/distance-matrix-asynchronous-example

- Example: https://docs.microsoft.com/en-us/bingmaps/rest-services/examples/distance-matrix-example

The HTTP GET request works as follow:

https://dev.virtualearth.net/REST/v1/Routes/DistanceMatrix?origins={lat0,long0;lat1,lon1;latM,lonM}&destinations={lat0,lon0;lat1,lon1;latN,longN}&travelMode={travelMode}&startTime={startTime}&timeUnit={timeUnit}&key={BingMapsKey}

- origins: list of lat, long coordinates of origins
- destionations: list of lat, long coordinates of destinations
- travelMode: `Driving`
- startTime: `2021-11-15-07:00`
- timeUnit
- BingMapsKey

In [3]:
def get_data(orig, dest, size_x, size_y):
    # Your Bing Maps Key
    bingMapsKey = "Ah52Budr_8V_sE1HKdPH-INIK_RDqas1zgkSJwdtB3GW3Dm04DsNm3lzkoXmpPc9"

    # input information
    travelMode = "Driving"
    startTime = "2021-11-15-07:00"
    distanceUnit = "km"
    timeUnit = "minute"

    routeUrl = (
        "https://dev.virtualearth.net/REST/v1/Routes/DistanceMatrix?"
        + f"origins={orig}"
        + f"&destinations={dest}"
        + f"&travelMode={travelMode}"
        + f"&startTime={startTime}"
        + f"&timeUnit={timeUnit}"
        + f"&key={bingMapsKey}"
    )

    # request = urllib.request.Request(routeUrl)
    # response = urllib.request.urlopen(request, proxies={"http":"http://myproxy:3129"})
    # data = json.load(response)

    response = requests.get(routeUrl)
    data = response.json()

    distance, time = build_od_matrix(data, (size_x, size_y))
    return distance, time

In [4]:
def build_od_matrix(data, size):

    resources = data["resourceSets"][0]["resources"][0]["results"]

    distance = np.zeros((size[0], size[1]))
    time = np.zeros((size[0], size[1]))

    for i in range(size[0]):
        for j in range(size[1]):
            distance[i, j] = resources[i * size[1] + j]["travelDistance"]
            time[i, j] = resources[i * size[1] + j]["travelDuration"]

    return distance, time
    

In [10]:
# Read the dataframe
df = pd.read_csv("../HousingData.csv")
df.head()

,Residence,Address,Lat,Long,Type,Capacity
0,Baker House,"362 Memorial Drive Building W7 Cambridge, MA 0213",42.356768,-71.095697,Undergrad,325
1,Burton Conner,"410 Memorial Drive Building W51 Cambridge, MA ...",42.356145,-71.097894,Undergrad,388
2,East Campus,"42.3600082317933, -71.08839675920484",42.360036,-71.088397,Undergrad,383
3,MacGregor House,"450 Memorial Drive Building W61 Cambridge, MA ...",42.355645,-71.099563,Undergrad,313
4,Maseeh Hall,"305 Memorial Drive Building W1 Cambridge, MA 0...",42.357843,-71.093448,Undergrad,500


In [8]:
# Matrix Initialization
distance_matrix = np.zeros((20, 20))
time_matrix = np.zeros((20, 20))

In [11]:
# Compute the list of origin-destinations points
long = list(df["Long"])
lat = list(df["Lat"])

points = []
for i, j in zip(lat, long):
    points.append(str(i) + "," + str(j))

In [ ]:
N = 25
Nn = math.ceil(145/N)

querry=False

if querry:

    ## Querying BingMaps Services
    for i in tqdm(range(Nn)):  # division in 6 batches
        min_, max_ = int(i * N), int(min(145, (i + 1) * N))
        origins = ";".join(points[min_:max_])

        for j in range(Nn):
            # ti.sleep(5)

            min_j, max_j = int(j * N), int((j + 1) * N)
            destinations = ";".join(points[min_j:max_j])

            distance, time = get_data(
                origins, destinations, size_x=int(max_ - min_), size_y=int(max_j - min_j)
            )

            # print(min_, max_, min_j, max_j)

            distance_matrix[min_:max_, min_j:max_j] = distance
            time_matrix[min_:max_, min_j:max_j] = time

    np.savetxt("../todo.txt", distance_matrix, fmt="%1.3f")

In [ ]:
if querry:
    dists = pd.DataFrame(distance_matrix, columns=sectors.names)
    dists["origin"] = sectors.names
    dists.set_index("origin", inplace=True)

    times = pd.DataFrame(time_matrix, columns=sectors.names)
    times["origin"] = sectors.names
    times.set_index("origin", inplace=True)
    
    #Save data
    dists.to_csv("../data/todo.csv")
    times.to_csv("../data/final_data/todo.csv")